In [1]:
import numpy as np
import glob
import pandas as pd
import matplotlib.pyplot as plt
import csv
import math

In [2]:
# Constants
TARGET_COUNT = 7
CIRCLE_RADIUS = 0.2
POLYLINE_SEGMENTS = [
    [np.array([0.2, 0.2]), np.array([0.2, -0.2])],
    [np.array([0.2, -0.2]), np.array([-0.2, -0.2])],
    [np.array([-0.2, -0.2]), np.array([-0.2, 0.2])],
    [np.array([-0.2, 0.2]), np.array([0.2, 0.2])],
]
OVERLAP_THRESHOLD = 0.03

In [5]:
# identification 
pid = 15

In [6]:
def getRelevantFiles(pid, ftype, task):
    return glob.glob("logs/" + str(pid) + "/" + ftype + "_" + str(pid) + "_" + task + "*")

In [7]:
def parseHeader(headerStr):
    info = headerStr.split(",")
    params = []
    if len(info) > 7:
        params = info[7:]
    return {
        "task start time (unix)": float(info[1]),
        "task start time (experiment)": float(info[2]),
        "tid": int(info[3]),
        "cid": int(info[4]),
        "task": info[5].lower(),
        "controller": info[6].lower(),
        "parameters": params
    }

In [8]:
def converttid(tid):
    taskid = math.ceil(float(tid) / 2)
    trialid = ((tid - 1) % 2) + 1
    return taskid, trialid

In [9]:
def convertController(inputType):
    info = inputType.split("_")
    controller = info[0].lower()
    technique = info[1].lower()
    return technique, controller

In [8]:
# Comparison
def parseComparisonLog(fname):
    f = open(fname).read().split("\n")
    taskInfo = parseHeader(f[0])
    f = f[1:]
    results = []
    for entry in f:
        info = entry.split(",")
        if info[0] != "COMPARISON":
            continue
        technique, controller = convertController(info[3])
        comparison = {
            "input technique": technique,
            "task": info[2].lower(),
            "controller preference": controller
        }
        results.append(comparison)
    results = pd.DataFrame(results)
    results.insert(0, "task start time (unix)", taskInfo["task start time (unix)"])
    results.insert(1, "task start time (experiment)", taskInfo["task start time (experiment)"])
    return results

def parseComparisonLogs():
    results = pd.DataFrame()
    fnames = getRelevantFiles(pid, "Interactions", "COMPARISON")
    for fname in fnames: 
        result = parseComparisonLog(fname)
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.insert(1, "task id", 0)
    results.to_csv("results/comparisons.csv", index=False)
    return results
    
parseComparisonLogs()

,participant id,task id


In [9]:
# Ratings
def parseRatingLog(fname):
    f = open(fname).read().split("\n")
    taskInfo = parseHeader(f[0])
    taskid, trialid = converttid(taskInfo["tid"])
    f = f[1:]
    results = []
    for entry in f: 
        info = entry.split(",")
        if info[0] != "RATING":
            continue
        technique, controller = convertController(info[4])
        rating = {
            "input technique": technique,
            "controller": controller,
            "task": info[3].lower(),
            "attribute": info[5].lower(),
            "rating": int(info[6])
        }
        results.append(rating)
    results = pd.DataFrame(results)
    results.insert(0, "controller id", taskInfo["cid"])
    results.insert(1, "task id", taskid)
    results.insert(2, "trial id", trialid)
    results.insert(3, "task start time (unix)", taskInfo["task start time (unix)"])
    results.insert(4, "task start time (experiment)", taskInfo["task start time (experiment)"])
    return results

def parseRatingLogs():
    results = pd.DataFrame()
    fnames = getRelevantFiles(pid, "Interactions", "RATING")
    for fname in fnames: 
        result = parseRatingLog(fname)
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/ratings.csv", index=False)
    return results 

parseRatingLogs()

,participant id,controller id,task id,trial id,task start time (unix),task start time (experiment),input technique,controller,task,attribute,rating
0,8,1,1,1,1.652703e+09,613.844,raycast,controller,trajectory_tracing,exertion,6
1,8,1,1,1,1.652703e+09,613.844,raycast,controller,trajectory_tracing,agency,6
0,8,2,1,1,1.652705e+09,2281.989,raycast,hand,trajectory_tracing,exertion,6
1,8,2,1,1,1.652705e+09,2281.989,raycast,hand,trajectory_tracing,agency,4
0,8,1,1,2,1.652704e+09,1046.391,raycast,controller,trajectory_tracing,exertion,6
1,8,1,1,2,1.652704e+09,1046.391,raycast,controller,trajectory_tracing,agency,5
0,8,2,1,2,1.652705e+09,2687.213,raycast,hand,trajectory_tracing,exertion,6
1,8,2,1,2,1.652705e+09,2687.213,raycast,hand,trajectory_tracing,agency,5
0,8,1,2,1,1.652704e+09,1476.080,raycast,controller,selection,exertion,8
1,8,1,2,1,1.652704e+09,1476.080,raycast,controller,selection,agency,6


In [10]:
def parsePose(pose):
    if type(pose) is str:
        pose = pose.split(",")
    return {
        "position": np.array(pose[0:3]).astype(float),
        "forward": np.array(pose[3:6]).astype(float),
        "up": np.array(pose[6:9]).astype(float),
        "rotation": np.array(pose[9:13]).astype(float),
        "relative position": np.array(pose[13:16]).astype(float),
        "relative forward": np.array(pose[16:19]).astype(float),
        "relative up": np.array(pose[19:22]).astype(float),
        "relative rotation": np.array(pose[22:]).astype(float)
    }

In [11]:
def parseUserVariables(userStr):
    info = userStr.split(",")
    scaleFactor = float(info[12])
    return scaleFactor

In [12]:
# Selection
def parseSelectionLog(fname):
    f = open(fname).read().split("\n")
    taskInfo = parseHeader(f[0])
    taskid, trialid = converttid(taskInfo["tid"])
    #taskPose = parsePose(taskInfo["parameters"])
    scaleFactor = parseUserVariables(open(glob.glob("logs/" + str(pid) + "\\User" + fname[len("logs/" + str(pid) + "\\Interactions"):])[0]).read().split("\n")[0])
    f = f[1:]
    currTrial = None
    results = []
    for entry in f:
        info = entry.split(",")
        if info[0] == "TRIAL":
            currTrial = {
                    "trial start time (unix)": float(info[1]),
                    "trial start time (experiment)": float(info[2]),
                    "trial start time (task)": float(info[3]),
                    "target number": int(info[4]),
                    "target id": int(info[8]),
                    "target position": np.array(info[5:8]).astype(float)
                }
        if info[0] == "SELECT" and currTrial != None:
            selectionPosition = np.array(info[5:]).astype(float)
            offset = selectionPosition - currTrial["target position"]
            selection = {
                "target number": currTrial["target number"],
                "target id": currTrial["target id"],
                "target position x": currTrial["target position"][0],
                "target position y": currTrial["target position"][1],
                "target position z": currTrial["target position"][2],
                "scaled target position x": scaleFactor * currTrial["target position"][0],
                "scaled target position y": scaleFactor * currTrial["target position"][1],
                "scaled target position z": scaleFactor * currTrial["target position"][2],
                "time (unix)": float(info[1]),
                "time (experiment)": float(info[2]),
                "time (task)": float(info[3]),
                "trial start time (unix)": currTrial["trial start time (unix)"],
                "trial start time (experiment)": currTrial["trial start time (experiment)"],
                "trial start time (task)": currTrial["trial start time (task)"],
                "selection time (unix)": float(info[1]) - currTrial["trial start time (unix)"],
                "selection time (experiment)": float(info[2]) - currTrial["trial start time (experiment)"],
                "selection time (task)": float(info[3]) - currTrial["trial start time (task)"],
                "success": eval(info[4]),
                "user position x": selectionPosition[0],
                "user position y": selectionPosition[1],
                "user position z": selectionPosition[2],
                "scaled user position x": scaleFactor * selectionPosition[0],
                "scaled user position y": scaleFactor * selectionPosition[1],
                "scaled user position z": scaleFactor * selectionPosition[2],
                "offset magnitude": np.linalg.norm(offset[:2]),
                "offset x": offset[0],
                "offset y": offset[1],
                "scaled offset magnitude": scaleFactor * np.linalg.norm(offset[:2]),
                "scaled offset x": scaleFactor * offset[0],
                "scaled offset y": scaleFactor * offset[1],
            }
            results.append(selection)
    results = pd.DataFrame(results)        
    results.insert(0, "controller id", taskInfo["cid"])
    results.insert(1, "task id", taskid)
    results.insert(2, "trial id", trialid)
    technique, controller = convertController(taskInfo["controller"])
    results.insert(3, "input technique", technique)
    results.insert(4, "controller", controller)
    results.insert(5, "scale factor", scaleFactor)
    results.insert(6, "task start time (unix)", taskInfo["task start time (unix)"])
    results.insert(7, "task start time (experiment)", taskInfo["task start time (experiment)"])
    return results
    
def parseSelectionLogs():
    results = pd.DataFrame()
    fnames = getRelevantFiles(pid, "Interactions", "SELECTION")
    results = pd.DataFrame()
    for fname in fnames: 
        result = parseSelectionLog(fname)
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/selections.csv", index=False)
    return results 

# Note: we can retrieve the task pose (commented out) and scale factor (from the user log file)

parseSelectionLogs()

,participant id,controller id,task id,trial id,input technique,controller,scale factor,task start time (unix),task start time (experiment),target number,...,user position z,scaled user position x,scaled user position y,scaled user position z,offset magnitude,offset x,offset y,scaled offset magnitude,scaled offset x,scaled offset y
0,8,1,2,1,raycast,controller,2.591887,1.652704e+09,1354.408,1,...,-0.006001,0.042520,0.472644,-0.015553,0.024093,0.016405,-0.017645,0.062447,0.042520,-0.045734
1,8,1,2,1,raycast,controller,2.591887,1.652704e+09,1354.408,2,...,-0.005992,-0.187485,-0.528689,-0.015530,0.027826,0.014441,-0.023785,0.072121,0.037430,-0.061648
2,8,1,2,1,raycast,controller,2.591887,1.652704e+09,1354.408,3,...,-0.000557,0.492064,0.435412,-0.001443,0.054729,0.033481,0.043292,0.141850,0.086780,0.112208
3,8,1,2,1,raycast,controller,2.591887,1.652704e+09,1354.408,3,...,-0.005998,0.386530,0.341389,-0.015547,0.010079,-0.007235,0.007017,0.026123,-0.018753,0.018186
4,8,1,2,1,raycast,controller,2.591887,1.652704e+09,1354.408,4,...,-0.005983,-0.457132,-0.125436,-0.015507,0.019018,0.018615,-0.003892,0.049292,0.048248,-0.010086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,8,2,2,2,raycast,hand,2.563303,1.652706e+09,3329.534,240,...,-0.004750,-0.325394,-0.540197,-0.012177,0.050344,-0.039928,-0.030664,0.129047,-0.102347,-0.078601
276,8,2,2,2,raycast,hand,2.563303,1.652706e+09,3329.534,240,...,-0.009904,-0.166751,-0.382371,-0.025386,0.037916,0.021962,0.030907,0.097189,0.056296,0.079224
277,8,2,2,2,raycast,hand,2.563303,1.652706e+09,3329.534,241,...,-0.009672,0.390871,0.358826,-0.024792,0.016014,-0.004045,0.015495,0.041050,-0.010367,0.039719
278,8,2,2,2,raycast,hand,2.563303,1.652706e+09,3329.534,242,...,-0.009720,-0.529519,-0.158340,-0.024915,0.020980,-0.011532,-0.017527,0.053779,-0.029560,-0.044926


In [13]:
def summarizeSelectionLogs():
    fnames = getRelevantFiles(pid, "Interactions", "SELECTION")
    results = []
    for fname in fnames: 
        result = parseSelectionLog(fname)
        print(result.keys())
        selectNum = len(result[result["success"] == True])
        errorNum = len(result[result["success"] == False])
        summary = {
            "controller id": result["controller id"][0],
            "task id": result["task id"][0],
            "trial id": result["trial id"][0],
            "input technique": result["input technique"][0],
            "controller": result["controller"][0],
            "task start time (unix)": result["task start time (unix)"][0],
            "task start time (experiment)": result["task start time (experiment)"][0],
            "mean selection time (unix)": np.mean(result["selection time (unix)"]),
            "mean selection time (experiment)": np.mean(result["selection time (experiment)"]),
            "mean selection time (task)": np.mean(result["selection time (task)"]),
            "number of selections": selectNum, 
            "number of errors": errorNum, 
            "error rate": float(errorNum) / float(selectNum + errorNum),
            "mean offset magnitude": np.mean(result["offset magnitude"]),
            "mean scaled offset magnitude": np.mean(result["scaled offset magnitude"])
        }
        results.append(summary)
    results = pd.DataFrame(results)
    results.insert(0, "participant id", pid)
    results.to_csv("results/selectionsSummary.csv", index=False)
    return results

summarizeSelectionLogs()

Index(['controller id', 'task id', 'trial id', 'input technique', 'controller',
       'scale factor', 'task start time (unix)',
       'task start time (experiment)', 'target number', 'target id',
       'target position x', 'target position y', 'target position z',
       'scaled target position x', 'scaled target position y',
       'scaled target position z', 'time (unix)', 'time (experiment)',
       'time (task)', 'trial start time (unix)',
       'trial start time (experiment)', 'trial start time (task)',
       'selection time (unix)', 'selection time (experiment)',
       'selection time (task)', 'success', 'user position x',
       'user position y', 'user position z', 'scaled user position x',
       'scaled user position y', 'scaled user position z', 'offset magnitude',
       'offset x', 'offset y', 'scaled offset magnitude', 'scaled offset x',
       'scaled offset y'],
      dtype='object')
Index(['controller id', 'task id', 'trial id', 'input technique', 'controller',
 

,participant id,controller id,task id,trial id,input technique,controller,task start time (unix),task start time (experiment),mean selection time (unix),mean selection time (experiment),mean selection time (task),number of selections,number of errors,error rate,mean offset magnitude,mean scaled offset magnitude
0,8,1,2,1,raycast,controller,1.652704e+09,1354.408,0.431936,0.431761,0.431718,244,32,0.115942,0.023185,0.060092
1,8,2,2,1,raycast,hand,1.652706e+09,2951.133,0.464630,0.464234,0.464261,231,42,0.153846,0.027102,0.070037
2,8,1,2,2,raycast,controller,1.652704e+09,1752.737,0.396928,0.396892,0.396913,268,38,0.124183,0.023770,0.061143
3,8,2,2,2,raycast,hand,1.652706e+09,3329.534,0.432223,0.431757,0.431735,243,37,0.132143,0.025651,0.065751


In [15]:
def parseTrajectory(trajectoryStr):
    info = trajectoryStr.split(",")
    return {
        "direction number": int(info[4]),
        "shape": info[5].lower(),
        "direction": info[6].lower(),
    }

def getProjectedDistanceCircle(position):
    distanceFromCenter = np.linalg.norm(position)
    direction = position / distanceFromCenter
    offset = position - (CIRCLE_RADIUS * direction)
    return offset

def pnt2line(pnt, start, end):
    lineVec = end - start 
    pntVec = pnt - start
    lineLen = np.linalg.norm(lineVec)
    lineUnitVec = lineVec / lineLen 
    pntVecScaled = pntVec / lineLen
    t = np.dot(lineUnitVec, pntVecScaled)   
    if t < 0.0:
        t = 0.0
    elif t > 1.0:
        t = 1.0
    nearest = t * lineVec 
    dist = np.linalg.norm(nearest - pntVec)
    nearest = start + nearest
    return (dist, nearest)

def getProjectedDistancePolyline(position):
    nearest = np.zeros(2)
    nearestDist = np.infty
    for segment in POLYLINE_SEGMENTS:
        dist, point = pnt2line(position, segment[0], segment[1])
        if dist < nearestDist: 
            nearest = point
            nearestDist = dist
    return position - nearest

# Trajectory tracing
def parseTrajectoryLog(fname):
    f = open(fname).read().split("\n")
    taskInfo = parseHeader(f[0])
    taskid, trialid = converttid(taskInfo["tid"])
    trajectoryInfo = parseTrajectory(f[1])
    scaleFactor = parseUserVariables(open(glob.glob("logs/" + str(pid) + "\\User" + fname[len("logs/" + str(pid) + "\\Interactions"):])[0]).read().split("\n")[0])
    f = f[2:]
    currTrace = {}
    touchNumber = 0 
    results = []
    for entry in f:
        info = entry.split(",")
        if info[0] == "START":
            touchNumber += 1
            currTrace = {
                "touch number": touchNumber, 
                "trace start time (unix)": float(info[1]),
                "trace start time (experiment)": float(info[2]),
                "trace start time (task)": float(info[3]),
                "trace start time (trace)": float(info[4])
            }
        if info[0] == "TRACING":
            targetPosition = np.array(info[5:8]).astype(float)
            userPosition = np.array(info[8:]).astype(float)
            offset = userPosition - targetPosition
            projectedOffset = None 
            if trajectoryInfo["shape"] == "circle":
                projectedOffset = getProjectedDistanceCircle(userPosition[:2])
            if trajectoryInfo["shape"] == "polyline":
                projectedOffset = getProjectedDistancePolyline(userPosition[:2])
            trace = {
                "touch number": currTrace["touch number"],
                "time (unix)": float(info[1]),
                "time (experiment)": float(info[2]),
                "time (task)": float(info[3]),
                "time (trace)": float(info[4]),
                "trace start time (unix)": currTrace["trace start time (unix)"],
                "trace start time (experiment)": currTrace["trace start time (experiment)"],
                "trace start time (task)": currTrace["trace start time (task)"],
                "trace start time (trace)": currTrace["trace start time (trace)"],
                "trace time (unix)": float(info[1]) - currTrace["trace start time (unix)"],
                "trace time (experiment)": float(info[2]) - currTrace["trace start time (experiment)"],
                "trace time (task)": float(info[3]) - currTrace["trace start time (task)"],
                "trace time (trace)": float(info[4]) - currTrace["trace start time (trace)"],
                "target position x": targetPosition[0],
                "target position y": targetPosition[1],
                "target position z": targetPosition[2],
                "user position x": userPosition[0],
                "user position y": userPosition[1],
                "user position z": userPosition[2],
                "real-time offset magnitude": np.linalg.norm(offset[:2]),
                "real-time offset x": offset[0],
                "real-time offset y": offset[1],
                "real-time overlapping": np.linalg.norm(offset[:2]) < OVERLAP_THRESHOLD,
                "projected offset magnitude": np.linalg.norm(projectedOffset[:2]),
                "projected offset x": projectedOffset[0],
                "projected offset y": projectedOffset[1],
                "projected overlapping": np.linalg.norm(projectedOffset[:2]) < OVERLAP_THRESHOLD,
                "scaled target position x": scaleFactor * targetPosition[0],
                "scaled target position y": scaleFactor * targetPosition[1],
                "scaled target position z": scaleFactor * targetPosition[2],
                "scaled user position x": scaleFactor * userPosition[0],
                "scaled user position y": scaleFactor * userPosition[1],
                "scaled user position z": scaleFactor * userPosition[2],
                "scaled real-time offset magnitude": scaleFactor * np.linalg.norm(offset[:2]),
                "scaled real-time offset x": scaleFactor * offset[0],
                "scaled real-time offset y": scaleFactor * offset[1],
                "scaled real-time overlapping": (scaleFactor * np.linalg.norm(offset[:2])) < OVERLAP_THRESHOLD,
                "scaled projected offset magnitude": scaleFactor * np.linalg.norm(projectedOffset[:2]),
                "scaled projected offset x": scaleFactor * projectedOffset[0],
                "scaled projected offset y": scaleFactor * projectedOffset[1],
                "scaled projected overlapping": (scaleFactor * np.linalg.norm(projectedOffset[:2])) < OVERLAP_THRESHOLD
            }
            results.append(trace)
    results = pd.DataFrame(results)  
    results.insert(0, "controller id", taskInfo["cid"])
    results.insert(1, "task id", taskid)
    results.insert(2, "trial id", trialid)
    technique, controller = convertController(taskInfo["controller"])
    results.insert(3, "input technique", technique)
    results.insert(4, "controller", controller)
    results.insert(5, "scale factor", scaleFactor)
    results.insert(6, "task start time (unix)", taskInfo["task start time (unix)"])
    results.insert(7, "task start time (experiment)", taskInfo["task start time (experiment)"])
    results.insert(8, "direction number", trajectoryInfo["direction number"])
    results.insert(9, "shape", trajectoryInfo["shape"])
    results.insert(10, "direction", trajectoryInfo["direction"])
    return results
    
def parseTrajectoryLogs():
    results = pd.DataFrame()
    fnames = getRelevantFiles(pid, "Interactions", "TRAJECTORY_TRACING")
    results = pd.DataFrame()
    for fname in fnames: 
        result = parseTrajectoryLog(fname)
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/trajectories.csv", index=False)
    return results 

#parseTrajectoryLogs()

In [18]:
def summarizeTrajectoryLogs():
    fnames = getRelevantFiles(pid, "Interactions", "TRAJECTORY_TRACING")
    results = []
    for fname in fnames: 
        result = parseTrajectoryLog(fname)
        results.append({
            "controller id": result["controller id"][0],
            "task id": result["task id"][0],
            "trial id": result["trial id"][0],
            "input technique": result["input technique"][0],
            "controller": result["controller"][0],
            "task start time (unix)": result["task start time (unix)"][0],
            "task start time (experiment)": result["task start time (experiment)"][0],
            "direction number": result["direction number"][0],
            "shape": result["shape"][0],
            "direction": result["direction"][0],
            "number of strokes": np.max(result["touch number"]),
            "mean real-time offset magnitude": np.mean(result["real-time offset magnitude"]),
            "mean projected offset magnitude": np.mean(result["projected offset magnitude"]),
            "projected overlap ratio": len(result[result["projected overlapping"] == True]) / len(result),
            "real-time overlap ratio": len(result[result["real-time overlapping"] == True]) / len(result),
            "mean scaled real-time offset magnitude": np.mean(result["scaled real-time offset magnitude"]),
            "mean scaled projected offset magnitude": np.mean(result["scaled projected offset magnitude"]),
            "scaled projected overlap ratio": len(result[result["scaled projected overlapping"] == True]) / len(result),
            "scaled real-time overlap ratio": len(result[result["scaled real-time overlapping"] == True]) / len(result),
            "tracing ratio": np.max(result["time (trace)"]) / np.max(result["time (task)"])
        })
    results = pd.DataFrame(results)
    results.insert(0, "participant id", pid)
    results.to_csv("results/trajectoriesSummary.csv", index=False)
    return results 

summarizeTrajectoryLogs()

,participant id,controller id,task id,trial id,input technique,controller,task start time (unix),task start time (experiment),direction number,shape,...,number of strokes,mean real-time offset magnitude,mean projected offset magnitude,projected overlap ratio,real-time overlap ratio,mean scaled real-time offset magnitude,mean scaled projected offset magnitude,scaled projected overlap ratio,scaled real-time overlap ratio,tracing ratio
0,8,1,1,1,raycast,controller,1.652703e+09,490.9651,1,circle,...,1,0.005660,0.003370,1.000000,1.000000,0.014597,0.008691,0.979299,0.948713,0.992252
1,8,1,1,1,raycast,controller,1.652703e+09,551.4348,2,circle,...,1,0.005080,0.003161,1.000000,1.000000,0.013102,0.008153,0.977065,0.959351,0.987146
2,8,2,1,1,raycast,hand,1.652705e+09,2158.2550,1,polyline,...,5,0.009050,0.005050,0.998322,0.995713,0.023435,0.013078,0.928798,0.738490,0.975833
3,8,2,1,1,raycast,hand,1.652705e+09,2219.7320,2,polyline,...,1,0.009164,0.004853,1.000000,0.994967,0.023732,0.012568,0.934762,0.721342,0.989005
4,8,1,1,2,raycast,controller,1.652704e+09,922.5851,1,polyline,...,1,0.006758,0.003282,1.000000,0.998882,0.017460,0.008478,0.963653,0.862815,0.978173
5,8,1,1,2,raycast,controller,1.652704e+09,983.9219,2,polyline,...,1,0.007242,0.004140,1.000000,0.997762,0.018710,0.010695,0.965691,0.874697,0.982286
6,8,2,1,2,raycast,hand,1.652705e+09,2562.1730,1,circle,...,1,0.006902,0.004566,1.000000,1.000000,0.017689,0.011702,0.954512,0.899888,0.964861
7,8,2,1,2,raycast,hand,1.652705e+09,2624.3540,2,circle,...,1,0.006917,0.004750,1.000000,1.000000,0.017727,0.012175,0.976696,0.917785,0.981447


In [12]:
def parseRAW(rawInfo):
    return {
        "acceleration": np.array(rawInfo[0:3]).astype(float),
        "angular acceleration": np.array(rawInfo[3:6]).astype(float),
        "velocity": np.array(rawInfo[6:9]).astype(float),
        "angular velocity": np.array(rawInfo[9:12]).astype(float),
        "position": np.array(rawInfo[12:15]).astype(float),
        "orientation": np.array(rawInfo[15:19]).astype(float), 
    }

# Logging tracked values
def parseTrackingLog(fname):
    f = open(fname).read().split("\n")
    taskInfo = parseHeader(f[0])
    taskid, trialid = converttid(taskInfo["tid"])
    f = f[1:]

    tLast = 0 
    tUnix = 0
    tExperiment = 0
    tTask = 0 
    epochs = []
    epoch = {}
    for entry in f: 
        info = entry.split(',')
        obj = info[0]
        if len(info) <= 1:
            continue
        tUnix = float(info[1])
        tExperiment = float(info[2])
        tTask = float(info[3])
        if tTask != tLast and tLast != 0:
            epochs.append(epoch)
            epoch = {}
        epoch["time (unix)"] = float(info[1])
        epoch["time (experiment)"] = float(info[2])
        epoch["time (task)"] = float(info[3])
        if obj == "RAW":
            rawValues = parseRAW(info[4:])
            epoch["raw acceleration x"] = rawValues["acceleration"][0]
            epoch["raw acceleration y"] = rawValues["acceleration"][1]
            epoch["raw acceleration z"] = rawValues["acceleration"][2]
            epoch["raw angular acceleration x"] = rawValues["angular acceleration"][0]
            epoch["raw angular acceleration y"] = rawValues["angular acceleration"][1]
            epoch["raw angular acceleration z"] = rawValues["angular acceleration"][2]
            epoch["raw velocity x"] = rawValues["velocity"][0]
            epoch["raw velocity y"] = rawValues["velocity"][1]
            epoch["raw velocity z"] = rawValues["velocity"][2]
            epoch["raw angular velocity x"] = rawValues["angular velocity"][0]
            epoch["raw angular velocity y"] = rawValues["angular velocity"][1]
            epoch["raw angular velocity z"] = rawValues["angular velocity"][2]
            epoch["raw position x"] = rawValues["position"][0]
            epoch["raw position y"] = rawValues["position"][1]
            epoch["raw position z"] = rawValues["position"][2]
            epoch["raw orientation w"] = rawValues["orientation"][0]
            epoch["raw orientation x"] = rawValues["orientation"][1]
            epoch["raw orientation y"] = rawValues["orientation"][2]
            epoch["raw orientation z"] = rawValues["orientation"][3]
        elif obj == "CURSOR":
            cursorActive = eval(info[4])
            epoch["cursor active"] = cursorActive
            cursorPosition = np.zeros(3)
            cursorPosition[:] = np.NaN
            if cursorActive: 
                cursorPosition = np.array(info[5:]).astype(float)
            epoch["cursor position x"] = cursorPosition[0]
            epoch["cursor position y"] = cursorPosition[1]
            epoch["cursor position z"] = cursorPosition[2]
        elif obj == "PRESSURE":
            cursorActive = eval(info[4])
            isPinching = eval(info[5])
            pinchStrength = float(info[6])
            epoch["cursor active"] = cursorActive
            epoch["pinching"] = isPinching
            epoch["pinch strength"] = pinchStrength
        elif obj == "RULA":
            continue
        else: 
            pose = parsePose(info[4:])
            epoch[obj + " position x"] = pose["position"][0]
            epoch[obj + " position y"] = pose["position"][1]
            epoch[obj + " position z"] = pose["position"][2]
            epoch[obj + " forward x"] = pose["forward"][0]
            epoch[obj + " forward y"] = pose["forward"][1]
            epoch[obj + " forward z"] = pose["forward"][2]
            epoch[obj + " up x"] = pose["up"][0]
            epoch[obj + " up y"] = pose["up"][1]
            epoch[obj + " up z"] = pose["up"][2]
            epoch[obj + " rotation w"] = pose["rotation"][0]
            epoch[obj + " rotation x"] = pose["rotation"][1]
            epoch[obj + " rotation y"] = pose["rotation"][2]
            epoch[obj + " rotation z"] = pose["rotation"][3]
            epoch[obj + " relative position x"] = pose["relative position"][0]
            epoch[obj + " relative position y"] = pose["relative position"][1]
            epoch[obj + " relative position z"] = pose["relative position"][2]
            epoch[obj + " relative forward x"] = pose["relative forward"][0]
            epoch[obj + " relative forward y"] = pose["relative forward"][1]
            epoch[obj + " relative forward z"] = pose["relative forward"][2]
            epoch[obj + " relative up x"] = pose["relative up"][0]
            epoch[obj + " relative up y"] = pose["relative up"][1]
            epoch[obj + " relative up z"] = pose["relative up"][2]
            epoch[obj + " relative rotation w"] = pose["relative rotation"][0]
            epoch[obj + " relative rotation x"] = pose["relative rotation"][1]
            epoch[obj + " relative rotation y"] = pose["relative rotation"][2]
            epoch[obj + " relative rotation z"] = pose["relative rotation"][3]

        tLast = tTask 
    epochs = pd.DataFrame(epochs)
    epochs.insert(0, "controller id", taskInfo["cid"])
    epochs.insert(1, "task id", taskid)
    epochs.insert(2, "trial id", trialid)
    technique, controller = convertController(taskInfo["controller"])
    epochs.insert(3, "input technique", technique)
    epochs.insert(4, "controller", controller)
    epochs.insert(5, "task start time (unix)", taskInfo["task start time (unix)"])
    epochs.insert(6, "task start time (experiment)", taskInfo["task start time (experiment)"])
    
    return epochs

def parseSelectionTrackedLogs():
    fnames = getRelevantFiles(pid, "Tracked", "SELECTION")
    results = pd.DataFrame()
    for fname in fnames: 
        result = parseTrackingLog(fname)
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/" + str(pid) + "/trackedSelection.csv", index=False)
    return results 

def parseTrajectoryTrackedLogs():
    fnames = getRelevantFiles(pid, "Tracked", "TRAJECTORY_TRACING")
    results = pd.DataFrame()
    for fname in fnames: 
        result = parseTrackingLog(fname)
        trajectoryInfo = parseTrajectory(open(glob.glob("logs/" + str(pid) + "\\Interactions" + fname[len("logs/" + str(pid) + "\\Tracked"):])[0]).read().split("\n")[1])
        result.insert(5, "direction number", int(trajectoryInfo["direction number"]))
        result.insert(6, "shape", trajectoryInfo["shape"])
        result.insert(7, "direction", trajectoryInfo["direction"])
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/" + str(pid) + "/trackedTrajectoryTracing.csv", index=False)
    return results 

In [19]:
for i in range(17,25):
    pid = i
    parseSelectionTrackedLogs()
    parseTrajectoryTrackedLogs()

In [23]:
def parseRulaLog(fname):
    f = open(fname).read().split("\n")
    taskInfo = parseHeader(f[0])
    taskid, trialid = converttid(taskInfo["tid"])
    epochs = []
    for entry in f: 
        info = entry.split(',')
        if info[0] == "RULA":
            epochs.append({
                "time (unix)": float(info[1]),
                "time (experiment)": float(info[2]),
                "time (task)": float(info[3]),
                "shoulder": float(info[4]),
                "elbow up/down": float(info[5]),
                "elbow left/right": float(info[6]),
                "wrist up/down": float(info[7]),
                "wrist left/right": float(info[8]),
                "wrist twist": float(info[9])
            })
    epochs = pd.DataFrame(epochs)
    epochs.insert(0, "controller id", taskInfo["cid"])
    epochs.insert(1, "task id", taskid)
    epochs.insert(2, "trial id", trialid)
    technique, controller = convertController(taskInfo["controller"])
    epochs.insert(3, "input technique", technique)
    epochs.insert(4, "controller", controller)
    epochs.insert(5, "task start time (unix)", taskInfo["task start time (unix)"])
    epochs.insert(6, "task start time (experiment)", taskInfo["task start time (experiment)"])
    return epochs

def parseSelectionRulaLogs():
    fnames = getRelevantFiles(pid, "Tracked", "SELECTION")
    results = pd.DataFrame()
    for fname in fnames: 
        result = parseRulaLog(fname)
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/rulaSelection.csv", index=False)
    return results 

def parseTrajectoryRulaLogs():
    fnames = getRelevantFiles(pid, "Tracked", "TRAJECTORY_TRACING")
    results = pd.DataFrame()
    for fname in fnames: 
        result = parseRulaLog(fname)
        trajectoryInfo = parseTrajectory(open(glob.glob("logs/" + str(pid) + "\\Interactions" + fname[len("logs/" + str(pid) + "\\Tracked"):])[0]).read().split("\n")[1])
        result.insert(5, "direction number", int(trajectoryInfo["direction number"]))
        result.insert(6, "shape", trajectoryInfo["shape"])
        result.insert(7, "direction", trajectoryInfo["direction"])
        results = pd.concat([results, result])
    results.insert(0, "participant id", pid)
    results.to_csv("results/rulaTrajectoryTracing.csv", index=False)
    return results 

In [24]:
parseSelectionRulaLogs()

,participant id,controller id,task id,trial id,input technique,controller,task start time (unix),task start time (experiment),time (unix),time (experiment),time (task),shoulder,elbow up/down,elbow left/right,wrist up/down,wrist left/right,wrist twist
0,8,1,2,1,raycast,controller,1.652704e+09,1354.408,1.652704e+09,1354.423,0.015356,-2.394237,-54.68977,-21.18045,-76.65001,55.04366,-21.36268
1,8,1,2,1,raycast,controller,1.652704e+09,1354.408,1.652704e+09,1354.432,0.024402,-2.383461,-54.68731,-21.18290,-76.57828,54.97193,-21.19450
2,8,1,2,1,raycast,controller,1.652704e+09,1354.408,1.652704e+09,1354.441,0.033329,-2.354896,-54.65580,-21.21441,-76.46265,54.85630,-21.02091
3,8,1,2,1,raycast,controller,1.652704e+09,1354.408,1.652704e+09,1354.453,0.045795,-2.413818,-54.60645,-21.26376,-76.72739,55.12104,-21.65030
4,8,1,2,1,raycast,controller,1.652704e+09,1354.408,1.652704e+09,1354.465,0.057037,-2.393304,-54.59217,-21.27804,-76.77682,55.17048,-21.70544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10720,8,2,2,2,raycast,hand,1.652706e+09,3329.534,1.652706e+09,3449.500,119.961900,18.073500,-122.24930,48.58225,-69.82321,87.37747,-60.07191
10721,8,2,2,2,raycast,hand,1.652706e+09,3329.534,1.652706e+09,3449.513,119.975300,17.992570,-122.27370,48.60665,-69.89401,87.44827,-60.02905
10722,8,2,2,2,raycast,hand,1.652706e+09,3329.534,1.652706e+09,3449.522,119.984400,17.852810,-122.26320,48.59613,-70.03898,87.59324,-60.57082
10723,8,2,2,2,raycast,hand,1.652706e+09,3329.534,1.652706e+09,3449.536,119.998000,17.726390,-122.31590,48.64883,-70.12268,87.67694,-60.23600


In [25]:
parseTrajectoryRulaLogs()

,participant id,controller id,task id,trial id,input technique,controller,direction number,shape,direction,task start time (unix),task start time (experiment),time (unix),time (experiment),time (task),shoulder,elbow up/down,elbow left/right,wrist up/down,wrist left/right,wrist twist
0,8,1,1,1,raycast,controller,1,circle,anticlockwise,1.652703e+09,490.9651,1.652703e+09,490.9816,0.016468,16.59476,-66.44924,-3.512960,-80.74121,57.53616,-9.839702
1,8,1,1,1,raycast,controller,1,circle,anticlockwise,1.652703e+09,490.9651,1.652703e+09,490.9921,0.026932,16.59169,-66.35428,-3.607923,-80.75960,57.55455,-9.700913
2,8,1,1,1,raycast,controller,1,circle,anticlockwise,1.652703e+09,490.9651,1.652703e+09,490.9990,0.033899,16.62883,-66.26659,-3.695603,-80.80640,57.60135,-9.896377
3,8,1,1,1,raycast,controller,1,circle,anticlockwise,1.652703e+09,490.9651,1.652703e+09,491.0115,0.046361,16.63264,-66.30945,-3.652751,-81.10250,57.89745,-10.585430
4,8,1,1,1,raycast,controller,1,circle,anticlockwise,1.652703e+09,490.9651,1.652703e+09,491.0234,0.058334,16.63474,-66.19971,-3.762495,-81.12064,57.91559,-10.490030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5456,8,2,1,2,raycast,hand,2,circle,anticlockwise,1.652705e+09,2624.3540,1.652705e+09,2685.4320,61.086650,45.53392,-127.80010,49.609080,-79.57058,101.87890,-66.036160
5457,8,2,1,2,raycast,hand,2,circle,anticlockwise,1.652705e+09,2624.3540,1.652705e+09,2685.4450,61.100270,45.52663,-127.76970,49.578580,-79.54021,101.84850,-66.075360
5458,8,2,1,2,raycast,hand,2,circle,anticlockwise,1.652705e+09,2624.3540,1.652705e+09,2685.4550,61.109640,45.53543,-127.74070,49.549660,-79.46630,101.77460,-65.998940
5459,8,2,1,2,raycast,hand,2,circle,anticlockwise,1.652705e+09,2624.3540,1.652705e+09,2685.4650,61.119880,45.53357,-127.70300,49.511920,-79.44340,101.75170,-65.976450
